In [2]:
# Load Cleaned Data
from google.colab import files
uploaded = files.upload()


Saving cleaned_sales_data.csv to cleaned_sales_data.csv


In [16]:
import pandas as pd

# Cleaned file load karo
df = pd.read_csv("cleaned_sales_data.csv")

# Date column ko phir se datetime me convert karo with dayfirst
df["weekend_date"] = pd.to_datetime(df["weekend_date"], format="mixed", dayfirst=True)




# Confirm check
print(df.dtypes)




weekend_date    datetime64[ns]
channel                 object
brand                   object
category                object
sub_category            object
SerialNum                int64
quantity                 int64
dtype: object


**Train/Validation Split**

In [4]:
!pip install prophet


In [17]:
from prophet import Prophet
import joblib


**Model Training Loop**

In [18]:
from prophet import Prophet
import joblib

serial_nums = df["SerialNum"].unique()

train_end = "2024-05-31"
valid_end = "2024-08-31"

for serial in serial_nums:
    # Filter for this SerialNum
    df_serial = df[df["SerialNum"] == serial][["weekend_date", "quantity"]].copy()

    # Rename columns for Prophet
    df_serial.rename(columns={"weekend_date": "ds", "quantity": "y"}, inplace=True)

    # Always sort
    df_serial.sort_values("ds", inplace=True)

    # Split train/validation
    train = df_serial[df_serial["ds"] <= train_end]
    valid = df_serial[(df_serial["ds"] > train_end) & (df_serial["ds"] <= valid_end)]

    # Check types
    print(train.dtypes)

    # Prophet model
    m = Prophet(weekly_seasonality=True, yearly_seasonality=True)
    m.fit(train)

    # Save model
    joblib.dump(m, f"prophet_model_serial_{serial}.pkl")


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkzsvissd/g864w1_q.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkzsvissd/73_w6to9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=86018', 'data', 'file=/tmp/tmpkzsvissd/g864w1_q.json', 'init=/tmp/tmpkzsvissd/73_w6to9.json', 'output', 'file=/tmp/tmpkzsvissd/prophet_modelnnc6143q/prophet_model-20250704040304.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:03:04 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:03:04 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkz

ds    datetime64[ns]
y              int64
dtype: object
ds    datetime64[ns]
y              int64
dtype: object


04:03:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkzsvissd/nxd7asmq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkzsvissd/l8nm6mou.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=43666', 'data', 'file=/tmp/tmpkzsvissd/nxd7asmq.json', 'init=/tmp/tmpkzsvissd/l8nm6mou.json', 'output', 'file=/tmp/tmpkzsvissd/prophet_model6s90j3qu/prophet_model-20250704040305.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:03:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:03:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run

ds    datetime64[ns]
y              int64
dtype: object
ds    datetime64[ns]
y              int64
dtype: object


04:03:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkzsvissd/o2q8wai8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkzsvissd/9pgbu50d.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=55823', 'data', 'file=/tmp/tmpkzsvissd/o2q8wai8.json', 'init=/tmp/tmpkzsvissd/9pgbu50d.json', 'output', 'file=/tmp/tmpkzsvissd/prophet_model0oos_h2b/prophet_model-20250704040305.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:03:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:03:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


ds    datetime64[ns]
y              int64
dtype: object


** Validation Forecast and Accuracy**

In [19]:
# Create dataframe of future dates for validation
future_valid = valid[["ds"]].copy()

# Make forecast on validation period
forecast_valid = m.predict(future_valid)

# Merge actuals with forecast
results_valid = valid.merge(forecast_valid[["ds", "yhat"]], on="ds")

print(results_valid.head())

# Compute absolute error
results_valid["abs_error"] = abs(results_valid["yhat"] - results_valid["y"])

# Monthly accuracy
monthly_accuracy = (
    results_valid
    .groupby(results_valid["ds"].dt.to_period("M"))
    .apply(lambda x: 1 - (x["abs_error"].sum() / x["y"].sum()))
)

print(f"Monthly Accuracy (Validation) for SerialNum {serial}:")
print(monthly_accuracy)


          ds     y         yhat
0 2024-06-01  1246  1650.338420
1 2024-06-01  1246  1650.338420
2 2024-06-01   790  1650.338420
3 2024-06-01   790  1650.338420
4 2024-06-08   939  1353.532078
Monthly Accuracy (Validation) for SerialNum 4:
ds
2024-06    0.314457
2024-07    0.293139
2024-08    0.483989
Freq: M, dtype: float64


** Future Forecast (Sept–Nov)**

In [20]:
# Generate future dataframe for next 12 weeks
future = m.make_future_dataframe(periods=12, freq="W")

# Make forecast
forecast = m.predict(future)

# Only future predictions
forecast_table = forecast[["ds", "yhat"]].tail(12)

print(f"Forecast for Sept–Nov for SerialNum {serial}:")
print(forecast_table)


Forecast for Sept–Nov for SerialNum 4:
           ds         yhat
71 2024-05-26  1075.437523
72 2024-06-02  1353.634376
73 2024-06-09   980.595678
74 2024-06-16   512.302982
75 2024-06-23   693.830009
76 2024-06-30  1537.040820
77 2024-07-07  2263.870128
78 2024-07-14  2167.527976
79 2024-07-21  1374.185003
80 2024-07-28   648.667382
81 2024-08-04   535.433922
82 2024-08-11   861.092853


In [21]:
results_valid.to_csv(f"validation_forecast_serial_{serial}.csv", index=False)


In [22]:
forecast_table.to_csv(f"future_forecast_serial_{serial}.csv", index=False)


In [23]:
joblib.dump(m, f"prophet_model_serial_{serial}.pkl")


['prophet_model_serial_4.pkl']

In [26]:
from prophet import Prophet
import pandas as pd
import joblib

# (Yeh maan ke chal raha hoon df aapka cleaned aur weekend_date datetime me hai)

# Sirf ek SerialNum ke liye example
serial = 4

# Filter data
df_serial = df[df["SerialNum"] == serial]

# Prophet input
prophet_df = df_serial[["weekend_date", "quantity"]].rename(columns={"weekend_date": "ds", "quantity": "y"})

# Train Model
m = Prophet(weekly_seasonality=True, yearly_seasonality=True)
m.fit(prophet_df)

# Create future dataframe: 13 weeks (Sept–Nov)
future = m.make_future_dataframe(periods=13, freq="W")

# Forecast
forecast = m.predict(future)

# Sirf Sept–Nov forecast rows nikalna
forecast_future = forecast.tail(13)

# Save as CSV
forecast_future[["ds", "yhat", "yhat_lower", "yhat_upper"]].to_csv(f"future_forecast_serial_{serial}.csv", index=False)

# Download
from google.colab import files
files.download(f"future_forecast_serial_{serial}.csv")


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkzsvissd/pojh3pju.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkzsvissd/mkucgzws.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=79482', 'data', 'file=/tmp/tmpkzsvissd/pojh3pju.json', 'init=/tmp/tmpkzsvissd/mkucgzws.json', 'output', 'file=/tmp/tmpkzsvissd/prophet_model6rg294mr/prophet_model-20250704042734.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:27:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:27:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
# 1️⃣ Pehle confirm kar lo kaun kaun si files hain
import os

# List all files in current directory
print("Current directory files:")
print(os.listdir())

# 2️⃣ Agar sab files yahin hain to zip bana do
import zipfile

# Zip file ka naam
zip_filename = "forecast_models_and_outputs.zip"

# Kaun kaun si files dalni hain (yahan sab file names likh do)
files_to_zip = [
    "future_forecast_serial_1.csv",
    "future_forecast_serial_2.csv",
    "future_forecast_serial_3.csv",
    "future_forecast_serial_4.csv",
    "future_forecast_serial_4(1).csv",
    "future_forecast_serial_5.csv",
    "validation_forecast_serial_1.csv",
    "validation_forecast_serial_2.csv",
    "validation_forecast_serial_3.csv",
    "validation_forecast_serial_4.csv",
    "validation_forecast_serial_5.csv",
    "prophet_model_serial_1.pkl",
    "prophet_model_serial_2.pkl",
    "prophet_model_serial_3.pkl",
    "prophet_model_serial_4.pkl",
    "prophet_model_serial_5.pkl"
]

# Zip file create karna
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        if os.path.exists(file):
            zipf.write(file)
        else:
            print(f"⚠️ File not found, skipping: {file}")

print(f"✅ ZIP file created: {zip_filename}")


Current directory files:
['.config', 'prophet_model_serial_5.pkl', 'prophet_model_serial_4 (1).pkl', 'validation_forecast_serial_4.csv', 'prophet_model_serial_4.pkl', 'future_forecast_serial_4.csv', 'prophet_model_serial_2.pkl', 'cleaned_sales_data.csv', 'prophet_model_serial_3.pkl', 'prophet_model_serial_1.pkl', 'sample_data']
⚠️ File not found, skipping: future_forecast_serial_1.csv
⚠️ File not found, skipping: future_forecast_serial_2.csv
⚠️ File not found, skipping: future_forecast_serial_3.csv
⚠️ File not found, skipping: future_forecast_serial_4(1).csv
⚠️ File not found, skipping: future_forecast_serial_5.csv
⚠️ File not found, skipping: validation_forecast_serial_1.csv
⚠️ File not found, skipping: validation_forecast_serial_2.csv
⚠️ File not found, skipping: validation_forecast_serial_3.csv
⚠️ File not found, skipping: validation_forecast_serial_5.csv
✅ ZIP file created: forecast_models_and_outputs.zip


In [28]:
from google.colab import files
files.download("forecast_models_and_outputs.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>